In [3]:
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker import image_uris
from sagemaker.estimator import Estimator

# Get SageMaker role
role = get_execution_role()

# Using Factorization machines algorithm from sagemaker image
container = image_uris.retrieve("factorization-machines", region="ap-northeast-2")

bucket = 'gcu-ml2-005-bucket'
sess = sagemaker.Session(
    default_bucket = bucket
)

fm = Estimator(container,
               role=role,
               instance_count=1,
               instance_type='ml.m5.large',
               sagemaker_session=sess)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [10]:
train_data = 's3://gcu-ml2-005-bucket/data/train.csv'
test_data = 's3://gcu-ml2-005-bucket/data/test.csv'

In [11]:
train_input = sagemaker.inputs.TrainingInput(train_data, content_type='text/csv')
test_input = sagemaker.inputs.TrainingInput(test_data, content_type='text/csv')


In [13]:
# 하이퍼파라미터 설정
feature_dim = 6  # 피처 차원
num_factors = 8  # FM 알고리즘의 요인 수 설정
mini_batch_size = 1000

# 하이퍼파라미터 설정 및 모델 학습
fm.set_hyperparameters(feature_dim=feature_dim,
                       num_factors=num_factors,
                       predictor_type='binary_classifier',
                       mini_batch_size=mini_batch_size)

fm.fit({'train': train_input, 'test': test_input})

INFO:sagemaker:Creating training-job with name: factorization-machines-2023-12-02-06-58-01-323


2023-12-02 06:58:01 Starting - Starting the training job...
2023-12-02 06:58:17 Starting - Preparing the instances for training......
2023-12-02 06:59:19 Downloading - Downloading input data......
2023-12-02 07:00:04 Training - Downloading the training image.....Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
[12/02/2023 07:01:13 INFO 140319266056000] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-conf.json: {'epochs': 1, 'mini_batch_size': '1000', 'use_bias': 'true', 'use_linear': 'true', 'bias_lr': '0.1', 'linear_lr': '0.001', 'factors_lr': '0.0001', 'bias_wd': '0.01', 'linear_wd': '0.001', 'factors_wd': '0.00001', 'bias_init_method': 'normal', 'bias_init_sigma': '0.01', 'linear_init_method': 'normal', 'linear_init_sigma

UnexpectedStatusException: Error for Training job factorization-machines-2023-12-02-06-58-01-323: Failed. Reason: ClientError: Rows 1-1000 in file /opt/ml/input/data/train/train.csv have different fields than the expected size 7., exit code: 2

INFO:sagemaker:Creating training-job with name: factorization-machines-2023-12-01-16-09-15-342


2023-12-01 16:09:15 Starting - Starting the training job...
2023-12-01 16:09:31 Starting - Preparing the instances for training.........
2023-12-01 16:10:47 Downloading - Downloading input data...
2023-12-01 16:11:23 Training - Downloading the training image..................
2023-12-01 16:14:44 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
[12/01/2023 16:14:51 INFO 140377310648128] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-conf.json: {'epochs': 1, 'mini_batch_size': '1000', 'use_bias': 'true', 'use_linear': 'true', 'bias_lr': '0.1', 'linear_lr': '0.001', 'factors_lr': '0.0001', 'bias_wd': '0.01', 'linear_wd': '0.001', 'factors_wd': '0.00001', 'bias

UnexpectedStatusException: Error for Training job factorization-machines-2023-12-01-16-09-15-342: Failed. Reason: ClientError: No value(s) were specified for 'feature_dim', 'num_factors', 'predictor_type' which are required hyperparameter(s) (caused by ValidationError), exit code: 2

In [10]:
data = pd.read_csv('./train_data.csv')

## Model NCF

In [18]:
%store -r n_user
%store -r n_item

print(n_user)
print(n_item)

20695
186


In [16]:
import os
import json
import sagemaker
import numpy as np
import pandas as pd
import tensorflow as tf
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

# get current SageMaker session's execution role and default bucket name
bucket = 'gcu-ml2-005-bucket'
sess = sagemaker.Session(
    default_bucket = bucket
)

role = get_execution_role()



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [14]:
# inspect the training script using `pygmentize` magic
!pygmentize 'ncf.py'

"""

 Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
 SPDX-License-Identifier: MIT-0
 
 Permission is hereby granted, free of charge, to any person obtaining a copy of this
 software and associated documentation files (the "Software"), to deal in the Software
 without restriction, including without limitation the rights to use, copy, modify,
 merge, publish, distribute, sublicense, and/or sell copies of the Software, and to
 permit persons to whom the Software is furnished to do so.

 THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
 INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
 PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
 HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION
 OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
 SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

"""


import tensorflow

In [19]:
# specify training instance type and model hyperparameters
# note that for the demo purpose, the number of epoch is set to 1

num_of_instance = 1                 # number of instance to use for training
instance_type = 'ml.m5.large'     # type of instance to use for training

training_script = 'ncf.py'

training_parameters = {
    'epochs': 1,
    'batch_size': 256, 
    'n_user': n_user, 
    'n_item': n_item
}

# training framework specs
tensorflow_version = '2.1.0'
python_version = 'py3'
distributed_training_spec = {'parameter_server': {'enabled': True}}

In [22]:

 training_data_uri = os.path.join(f's3://{bucket_name}', 'data') 
 print(training_data_uri)

In [24]:
# initiate the training job using Tensorflow estimator
ncf_estimator = TensorFlow(
    entry_point=training_script,
    role=role,
    instance_count=num_of_instance,
    instance_type=instance_type,
    framework_version=tensorflow_version,
    py_version=python_version,
    distribution=distributed_training_spec,
    hyperparameters=training_parameters
)

ncf_estimator.fit(train_data_uri)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Amazon SageMaker Debugger does not currently support Parameter Server distribution
INFO:sagemaker:Amazon SageMaker Debugger does not currently support Parameter Server distribution
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2023-12-02-07-11-19-744


Using provided s3_resource
2023-12-02 07:11:20 Starting - Starting the training job...
2023-12-02 07:11:39 Starting - Preparing the instances for training......
2023-12-02 07:12:43 Downloading - Downloading input data......
2023-12-02 07:13:38 Training - Training image download completed. Training in progress..2023-12-02 07:13:44,766 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2023-12-02 07:13:44,775 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-12-02 07:13:44,989 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-12-02 07:13:45,007 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-12-02 07:13:45,027 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-12-02 07:13:45,041 sagemaker-containers INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {
        "sagemaker_parameter_server_ena


2023-12-02 07:15:00 Completed - Training job completed
Training seconds: 138
Billable seconds: 138


In [25]:
endpoint_name = 'book-endpoint-ncf-005'
model_name = 'book-recommend-model-005'

predictor = ncf_estimator.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.large", 
    endpoint_name=endpoint_name,
    model_name=model_name,
)

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating model with name: book-recommend-model-005
INFO:sagemaker:Creating endpoint-config with name book-endpoint-ncf-005
INFO:sagemaker:Creating endpoint with name book-endpoint-ncf-005


---!

AttributeError: 'SageMaker' object has no attribute 'invoke_endpoint'